##### Source API for stock data: https://finnhub.io/

In [75]:
import requests

#### Simple wrapper class for API Calls:

In [76]:
class Finnhub_wrapper:
    
    def __init__(self):
        self.__base_url = 'https://finnhub.io/api/v1'
        self.candle_data_args = {
            'stock': {
                'required':  ['symbol', 'resolution'],
                'optional': ['from', 'to', 'format', 'adjusted'],
                'resolution': [1, 5, 15, 30, 60, 'D', 'W', 'M']
            },
            'crypto': {
                'required':  ['symbol', 'exchange', 'resolution'],
                'optional': ['from', 'to', 'format'],
                'exchange': ['KRAKEN', 'HITBTC', 'COINBASE', 'GEMINI', 'POLONIEX', 'BINANCE', 'ZB', 'BITTREX', 'KUCOIN', 'OKEX', 'BITFINEX', 'HUOBI'],
                'resolution': [1, 5, 15, 30, 60, 'D', 'W', 'M']
           }
        }
        
    # parameters: asset type (eg. 'stock', 'crypto') and arguments in dict form (eg. {'symbol':'BTCUSTDT', 'resolution': 'D', 'exchange':'BINANCE', 'from': 1572651390, 'to': 1575243390})
    def get_candle_data(self, asset, args):
        assert args is not None
        self.__check_args(asset, args)
        return requests.get(self.__construct_url(asset, args))
    
    def __check_args(self, asset, args):
        print(asset)
        if asset not in self.candle_data_args.keys():
            raise Exception('Provided type \'{}\' is not supported'.format(asset))
        for argument in self.candle_data_args[asset]['required']:
            if argument not in args.keys():
                raise Exception('Required argument is not provided. Argument {} needs to be provided'.format(argument))
        for argument in args.keys():
            if  (argument not in self.candle_data_args[asset]['required']) and (argument not in self.candle_data_args[asset]['optional']):
                raise Exception('Provided argument \'{}\' is not supported'.format(argument))
        if args['resolution'] not in self.candle_data_args[asset]['resolution']:
            raise Exception('Provided resolution \'{}\' is not supported. Supported resolutions: {}'.format(args['resolution'], self.candle_data_args[asset]['resolution']))
        if asset == 'crypto':
            if args['exchange'] not in self.candle_data_args[asset]['exchange']:
                raise Exception('Provided exchange \'{}\' is not supported'.format(args['exchange']))
            
    def __construct_url(self, asset, args):
        url = self.__base_url
        url += '/' + asset + '/' + 'candle?'
        url += 'symbol='
        if asset == 'crypto':
            url += args['exchange'] + ':'
        url += args['symbol'] + '&'
        url += 'resolution='
        if isinstance(args['resolution'], int):
            url += str(args['resolution'])
        else:
            url += args['resolution']
        for argument in self.candle_data_args[asset]['optional']:
            if ((argument+'=') not in url) and (argument in args.keys()):
                url += '&' + argument + '=' 
                if isinstance(args[argument], int):
                    url += str(args[argument])
                else:
                    url += args[argument]
        print(url)
        return url

In [77]:
# example queries
service = Finnhub_wrapper()
#print(service.get_candle_data('crypto', {'symbol':'BTCUSDT', 'resolution': 'D', 'exchange':'BINANCE', 'from': 1572651390, 'to': 1575243390}))
print(service.get_candle_data('stock', {'symbol':'AAPL', 'resolution': 1, 'from': 1572651390, 'to': 1572910590}).json())

stock
https://finnhub.io/api/v1/stock/candle?symbol=AAPL&resolution=1&from=1572651390&to=1572910590
<Response [200]>
